# Train Music Transformer
Since Fri. Feb. 25th, 2022

Set up training in colab


## Setup


### Ipython



In [5]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Colab



In [6]:
import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers==4.19.2 datasets
    ! pip3 install stefutils

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    # if torch.cuda.is_available():
    #     %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from stefutil import *
from musicnlp.util import *

mic(u.proj_dir, u.pkg_nm)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
eval  musicnlp	notebook  plot	tokenizers
Tue Jul 12 20:26:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 000000

ic| u.proj_dir: 'Symbolic-Music-Generation', u.pkg_nm: 'musicnlp'


('Symbolic-Music-Generation', 'musicnlp')

### code

In [7]:
from musicnlp.trainer import train


seed = sconfig('random-seed')



## Prep for training


In [ ]:
md_nm = 'reformer'
# md_sz = 'debug'
# md_sz = 'tiny'
md_sz = 'base'
mic(md_nm, md_sz)

# TODO: smaller seq-len for now, until it shows longer dependency
model_config = dict(max_position_embeddings=1024, axial_pos_shape=(32, 32))

n_ep = 16
train_args = dict(save_strategy='epoch', num_train_epochs=n_ep)

augment_key = False
wordpiece_tokenize = False
my_train_args = dict(
    tqdm=True, logging_strategy='epoch', save_epochs=1,
    augment_key=augment_key,
    wordpiece_tokenize=wordpiece_tokenize,
)

if 'debug' in md_sz or md_sz == 'tiny':
    train_args.update(dict(
        per_device_train_batch_size=4,
        num_train_epochs=64,
    ))
    my_train_args['save_epochs'] = 16
else:
    train_args.update(dict(
        learning_rate=1e-5,
        fp16=torch.cuda.is_available(),
        per_device_train_batch_size=64,
    ))

pop = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-04'
mst = 'musicnlp music extraction, dnm=MAESTRO, n=1276, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-28'
lmd = 'musicnlp music extraction, dnm=LMD, n=176640, meta={mode=melody, prec=5, th=1}, 2022-05-27_15-23-20'
dnms = [pop, mst, lmd]

# n = 64
n = None

checkpoint_path = os.path.join(u.model_path, '2022-07-12_00-09-14_reformer', 'checkpoint-16434')
mic(os.listdir(checkpoint_path))

mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, model_config=model_config,
    dataset_names=dnms, dataset_args=dict(n_sample=n, shuffle_seed=seed, pbar=True),
    train_args=train_args, my_train_args=my_train_args, trainer_args=dict(
        disable_train_metrics=True
    )
)



## Train


Check log and tensorboard files written



In [ ]:
# trainer.train()
# checkpoint_path = os.path.join(u.model_path, '2022-07-06_13-46-19_reformer', 'checkpoint-2739')
trainer.train(checkpoint_path)

save_path = os.path.join(trainer.args.output_dir, 'trained')
trainer.save_model(save_path)


In [ ]:
mic(trainer.args.output_dir)
mic(os.listdir(trainer.args.output_dir))

